In [2]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [122]:
def scrape_one_new(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('h1', class_='content--title').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

# for old web normal
def make_date_normal_old(hmd,time):
    year, month, day = hmd[:4], hmd[4:6], hmd[6:]
    hour, minute = time.split('時')
    minute = minute.strip('分')
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    return f"{year}-{month}-{day}T{hour}:{minute}"

def scrape_one_old(html, url):
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find('span', class_="contentTitle").text.strip()
    hmd_ = url.split('/')[-2]
    time_ = soup.find('span', id="news_time").text
    date = make_date_normal_old(hmd_, time_)
    genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB','ＮＨＫ','ＮＨＫニュース','']]
    article = soup.find('div', id="news_textbody").text
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':[],
        'url':url,
        'datePublished':date,
        'dateModified':""
    }

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

def getlink():
    notyet = []
    urllist = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    nolink = pd.read_csv('nolinknormal.txt', encoding='utf-8', header=None)[0].tolist()
    df_easy = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_easy['url_normal']:
        if i not in urllist:
            notyet.append(i)
    return sorted(set(notyet) - set(nolink))

def wrongid():
    df = pd.read_json('nhkwebeasy.json', encoding='utf-8')
    print(len(df))
    df['normalID'] = df['url_normal'].apply(lambda x:x.split('/')[-1].strip('.html'))
    return df[df['id'] != df['normalID']]['id'].tolist()

def wrongscrape():
    wrongids = wrongid()
    existurl = pd.read_csv('linknormal.txt', encoding='utf-8', header=None)[0].tolist()[::-1]
    for ID in wrongids:
        for url in existurl:
            if ID in url:
                print(url.split('/*/')[-1])
                break

In [131]:
wrongscrape()

3316


# scrape

In [4]:
driver = webdriver.Firefox()

In [128]:
url = 'https://www3.nhk.or.jp/news/html/20181206/k10011736251000.html'
driver.get(f'https://web.archive.org/web/*/{url}')
time.sleep(4)
html = str(driver.page_source.encode('utf-8'))
snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
driver.get('https://web.archive.org' + snap.group(2))

In [129]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
dic = scrape_one_old(html,url)
print(dic)

{'id': 'k10011736251000', 'title': '教職員４割が「過労死ライン」超の時間外と回答 日教組調査', 'article': '学校現場の働き方改革をめぐって、日教組＝日本教職員組合が行った意識調査で、４割近くの教職員が、いわゆる「過労死ライン」を超える１日４時間以上の時間外労働をしていると答えました。\n意識調査は、日教組がことし７月から９月にかけて、インターネットを通じて、全国の小学校や中学校などの教職員を対象に行い、１万1125人から回答を得ました。それによりますと、平日の１日の平均勤務時間を聞いたところ、97％の教職員が時間外労働をしていて、いわゆる「過労死ライン」とされる月80時間超えに相当する１日４時間以上の時間外労働をしていると答えた人は38％に上りました。また、週末の勤務状況を聞いたところ、59％の教職員が出勤していて、１日の平均出勤時間は２時間余りとなっています。一方、文部科学省が、平日の勤務時間を延長する代わりに、夏休みなどにまとまった休みを取る「変形労働時間制」の導入を検討するなか、夏休みの負担を軽減するために見直すべき業務を複数回答で尋ねたところ、「研究・研修会」が48％と最も多く、次いで「部活動指導」と「出張」が36％、「調査・報告」が35％などとなっています。日教組では「ほとんどの教職員が時間外労働をしている実態が明らかになった。今後の働き方改革の議論に生かしていきたい」と話しています。', 'genre': ['教育', '働き方改革', '社会'], 'keywords': [], 'url': 'https://www3.nhk.or.jp/news/html/20181206/k10011736251000.html', 'datePublished': '2018-12-06T04:11', 'dateModified': ''}


In [130]:
js(dic)